In [1]:
#please ensure to run all the cells in order to have the maps fully function!
import csv
import pandas as pd
import folium
from folium import plugins
from folium.plugins import HeatMap
import bqplot
from bqplot import pyplot as plt
from bqplot import Lines, Figure, LinearScale, DateScale, Axis, Scale, LogScale
from ipyleaflet import Map, GeoJSON, WidgetControl, SearchControl, LayerGroup, AwesomeIcon, Marker, Circle, FullScreenControl, LayersControl, basemaps
import os
import json
from ipywidgets import interact, interact_manual

In [2]:
def CSVtoLIST(file):
    dataList = []
    with open(file, newline='') as dataFile:
        csvReader = csv.reader(dataFile)
        next(csvReader, None)
        for line in csvReader:
            if line[0] == 'NA' or line[1] == 'NA':
                continue
            else:
                tempList = []
                tempList.insert(len(tempList), float(line[0]))
                tempList.insert(len(tempList), float(line[1]))
                tempList.insert(len(tempList), line[2])
                tempList.insert(len(tempList), line[3])
                tempList.insert(len(tempList), line[4])
                tempList.insert(len(tempList), line[5])
                dataList.insert(len(dataList), tempList)
    return dataList

def CSVtoListPrice(file):
    listWithPrices = []
    with open(file, newline='') as dataFile:
        csvReader = csv.reader(dataFile)
        next(csvReader, None)
        for line in csvReader:
            if line[0] == 'NA' or line[1] == 'NA':
                continue
            else:
                tempList = []
                tempList.insert(len(tempList), float(line[0]))
                tempList.insert(len(tempList), float(line[1]))
                if line[2] == 'NA':
                    tempList.insert(len(tempList), float(-1))
                else:
                    tempList.insert(len(tempList), float(line[2]))
                tempList.insert(len(tempList), line[3])
                tempList.insert(len(tempList), line[4])
                tempList.insert(len(tempList), line[5])
                listWithPrices.insert(len(listWithPrices), tempList)
    return listWithPrices

def listToDataFrame(dataList):
    dataDict = {'Lat': [], 'Lon': [], 'Infl_Price': [], 'Date': [], 'Neighborhood': [], 'DT': [], 'Year': []}
    for data in dataList:
        dataDict["Lat"].insert(len(dataDict["Lat"]), data[0])
        dataDict["Lon"].insert(len(dataDict["Lon"]), data[1])
        dataDict["Infl_Price"].insert(len(dataDict["Infl_Price"]), data[2])
        dataDict["Date"].insert(len(dataDict["Date"]), data[3])
        dataDict["Neighborhood"].insert(len(dataDict["Neighborhood"]), data[4])
        dataDict["DT"].insert(len(dataDict["DT"]), data[5])
        dataDict['Year'].insert(len(dataDict['Year']), int(data[3][0:4]))
    return dataDict

In [3]:
data = listToDataFrame(CSVtoLIST('Cleaned_Housing_Data.csv'))
df = pd.DataFrame(data=data)
priceData = listToDataFrame(CSVtoListPrice('Cleaned_Housing_Data.csv'))
priceDF = pd.DataFrame(data=priceData)

In [4]:
chartData = priceDF[(priceDF['Infl_Price'] > 1) & (priceDF['Year'] < 2021)]
lineData = chartData.groupby(['Neighborhood', 'Year']).size().reset_index()
fruitBelt = lineData[(lineData['Neighborhood'] == 'Fruit Belt') & (lineData['Year'] < 2021)]
fruitBelt = fruitBelt.rename(columns={0: 'Home'})

In [5]:
# import pandas as pd
# import re
# import folium
# from dateutil.parser import parse


# housingDataFrame = pd.read_csv('Market_Data.csv')
# housingDataFrame = housingDataFrame.dropna()
# locationColumn = housingDataFrame['LOCATION_open_data']
# locationList = []
# lat = []
# lon = []
# for location in locationColumn:
#     locationList.insert(len(locationList), re.sub('[()]', '', location))

# for location in locationList:
#     combo = location.split(", ")
#     lat.insert(len(lat), combo[0])
#     lon.insert(len(lon), combo[1])

# housingDataFrame['LAT'] = lat
# housingDataFrame['LON'] = lon

# housingDataDict = []
# for row in housingDataFrame.itertuples():
#     if row[3] != 'No Sales History' or row[7] != 'No Sales History' and row[3] != '$1':
#         housingDict = {'Date_Sold': row[2],
#                        'Price': row[3],
#                        'Total_Value': row[7],
#                        'Census_Tract': row[11],
#                        'Block_Group': row[12],
#                        'SBL': row[13],
#                        'Address': row[14],
#                        'Neighborhood': row[15],
#                        'Latitude': row[17],
#                        'Longitude': row[18],
#                        'Prior_Owner': row[6],
#                        'Year': 0
#                        }
#         housingDataDict.insert(len(housingDataDict), housingDict)

# for obj in housingDataDict:
#     obj['Price'] = re.sub('[$]', '', obj['Price'])
#     obj['Price'] = re.sub('[ ]', '', obj['Price'])
#     obj['Price'] = float(obj['Price'].replace(',', ''))
#     obj['Total_Value'] = re.sub('[$]', '', obj['Total_Value'])
#     obj['Total_Value'] = re.sub('[ ]', '', obj['Total_Value'])
#     obj['Total_Value'] = re.sub('[()]', '', obj['Total_Value'])
#     obj['Total_Value'] = float(obj['Total_Value'].replace(',', ''))

# for obj in housingDataDict:
#     date = str(parse(obj['Date_Sold']).date())
#     date = date[0:4]
#     obj['Year'] = int(date)

# housingDataDict = [i for i in housingDataDict if (i['Year'] <= 2020)]

C:\Users\KevinBrito\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
predictionData = pd.read_csv('prediction_Data.csv')
predictionDataDict = []
for row in predictionData.itertuples():
    tempDict = {'SBL': row[1],
                'ProbSell': row[2],
                'Neighborhood': row[3],
                'Lat': row[8],
                'Lon': row[9],
                'Bought': row[10],
                'ProbBought': row[11],
                'ProbNotBought': row[12]
    }
    predictionDataDict.insert(len(tempDict), tempDict)
predictionData

,SBL,Probability Of Selling,NEIGHBORHOOD,LOCATION,Census Code (GEOIDs),CENSUS TRACT,CENSUS BLOCK GROUP,LATITUDE,LONGITUDE,was bought 2019,Probability Of being bought 2019,Probability Of not being bought 2019
0,100.47-1-16,[0.6225 0.3775],Elmwood Bryant,"(42.909603712417976, -78.86975196686805)",3.602901e+11,67.02,1,42.909604,-78.869752,0,0.377500,0.622500
1,99.34-3-21,[0.58875 0.41125],Upper West Side,"(42.91538654263488, -78.89650544529762)",3.602901e+11,61,1,42.915387,-78.896505,0,0.411250,0.588750
2,99.34-3-22,[0.5775 0.4225],Upper West Side,"(42.91538812517063, -78.89659841929371)",3.602901e+11,61,1,42.915388,-78.896598,0,0.422500,0.577500
3,112.62-2-12,[0.6525 0.3475],Broadway Fillmore,"(42.88198144897082, -78.83027416733262)",3.602900e+11,17,1,42.881981,-78.830274,0,0.347500,0.652500
4,99.36-7-13.1,[0.49125 0.50875],Elmwood Bryant,"(42.91468397357915, -78.8836715007532)",3.602901e+11,66.01,1,42.914684,-78.883672,0,0.508750,0.491250
...,...,...,...,...,...,...,...,...,...,...,...,...
48451,99.42-9-6,[0.21625 0.78375],West Side,"(42.91255156650591, -78.89522029580017)",3.602901e+11,69.01,3,42.912552,-78.895220,1,0.783750,0.216250
48452,89.69-2-31,[0.25 0.75],Elmwood Bidwell,"(42.92499766211617, -78.8792602198118)",3.602901e+11,63.01,1,42.924998,-78.879260,1,0.750000,0.250000
48453,90.83-3-41,[0.22 0.78],Schiller Park,"(42.92091021621923, -78.80088116121952)",3.602900e+11,38,1,42.920910,-78.800881,1,0.780000,0.220000
48454,89.30-1-41,[0.25791667 0.74208333],Parkside,"(42.93807090964701, -78.87563992795813)",3.602901e+11,54,3,42.938071,-78.875640,1,0.742083,0.257917


In [7]:
chartData

priceData = chartData.groupby(['Neighborhood', 'Year']).Infl_Price.mean().reset_index()
priceDataFB = priceData[(priceData['Neighborhood'] == 'Fruit Belt') & (priceData['Year'] < 2021)]
priceDataBuff = chartData.groupby(['Year']).Infl_Price.mean().reset_index()

foreclosures = chartData[(chartData['DT'] == 'D1D') & (df['Year'] <= 2020)]
foreclosures = foreclosures.groupby(['Neighborhood', 'Year']).size().reset_index()
foreclosures = foreclosures.rename(columns={0: 'Foreclosures'})
foreclosures = foreclosures[(foreclosures['Neighborhood'] == 'Fruit Belt')]

C:\Users\KevinBrito\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [16]:
date_start = pd.datetime(1985, 12, 31)
date_end = pd.datetime(2020, 12, 31)

date_scale = DateScale(min=date_start, max=date_end)
date_data = pd.date_range(start=date_start, end=date_end, freq='A', normalize=True)

x_scale = LinearScale()

lines = Lines(x=priceDataFB.Year, y=priceDataFB.Infl_Price, scales={'x': LinearScale(), 'y': x_scale})
ax_x = Axis(label='Year', scale=date_scale, num_ticks=7, tick_format='%Y')
ax_y = Axis(label='Price', scale=x_scale, orientation='vertical', side='left', label_offset='50px')
averageHomePricePlot = Figure(axes=[ax_x, ax_y], title='Average Home Price in Fruit Belt', marks=[lines], animation_duration=500,
                layout={'max_height': '250px', 'max_width': '700px'})

averageHomePricePlot

Figure(animation_duration=500, axes=[Axis(label='Year', num_ticks=7, scale=DateScale(max=datetime.datetime(202…

In [9]:
#Find a start date and end date
date_start = pd.datetime(1994, 12, 31)
date_end = pd.datetime(2011, 12, 31)

#Find a scale for the data
date_scale = DateScale(min=date_start, max=date_end)
date_data = pd.date_range(start=date_start, end=date_end, freq='A', normalize=True)

x_scale = LinearScale()
data = Lines(x=foreclosures.Year, y=foreclosures.Foreclosures, scales={'x': LinearScale(), 'y': x_scale})
foreclosureAx_x = Axis(label='Year', scale=date_scale, num_ticks=7, tick_format='%Y')
foreclosureAx_y = Axis(label='Number of Foreclosures', scale=x_scale, orientation='vertical', side='left')
foreclosurePlot = Figure(axes=[foreclosureAx_x, foreclosureAx_y], title='Number of Foreclosures in Fruit Belt', marks=[data], animation_duration=500,
                layout={'max_height': '250px', 'max_width': '700px'})
foreclosurePlot

plotDict = {'Average Home Price': averageHomePricePlot, 'Foreclosure': foreclosurePlot}
dataDict = {'Average Home Price': priceData, 'Foreclosure': foreclosures}

def figure_change(plot, neighborhood_name, data, figure):
    if figure == 'Average Home Price':
        markY = Lines(x=priceDataFB.Year, y=data[(data['Neighborhood'] == neighborhood_name) & (data['Year'] < 2021)].Infl_Price, scales={'x': LinearScale(), 'y': x_scale})
        plot.title = 'Average Home Price in ' + neighborhood_name
        plot.marks = [markY]
        plot.axes[1].label = 'Price'
    elif figure == 'Foreclosure':
        markY = Lines(x=foreclosures.Year, y=data[(data['Neighborhood'] == neighborhood_name) & (data['Year'] < 2021)].Foreclosures, scales={'x': LinearScale(), 'y': x_scale})
        plot.title = 'Number of Foreclosures in ' + neighborhood_name
        plot.marks = [markY]
        plot.axes[1].label = 'Foreclosures'

figure_change(plotDict['Foreclosure'], 'Fruit Belt', dataDict['Foreclosure'], 'Foreclosure')
#change the figure THEN change the figure on display... figure_change allows us to change the figure that is already being displayed 
#but it does NOT allow us to choose WHICH figure to display.
foreclosurePlot

Figure(animation_duration=500, axes=[Axis(label='Year', num_ticks=7, scale=DateScale(max=datetime.datetime(201…

In [10]:
def swap_figure(widgets, Figures):
    plot = plotDict[Figures]
    if len(widgets) == 0:
        plotWidget = WidgetControl(widget=plot[Figures], position='bottomleft', max_width=700, max_height=700)
        basicMap.add_control(plotWidget)
        widgets.append(plotDict[Figures])
    else:
        plot = plotDict[Figures]


In [11]:
def update_figure(neighborhood_name, data):
    lines.y = data[(data['Neighborhood'] == neighborhood_name) & (data['Year'] < 2021)].Infl_Price
    ax_y.label = 'Price'
    averageHomePricePlot.title = "Average Home Price in " + neighborhood_name

update_figure('Fruit Belt', priceData)

In [17]:
#please use this cell to demonstrate on hover and on click effects!
center = [42.880230, -78.878738]
dynamicMap = Map(center=center, zoom=11)

marker = Marker(icon=AwesomeIcon(name="check", marker_color='green', icon_color='darkgreen'))

search = SearchControl(position="topright", 
                       url='https://nominatim.openstreetmap.org/search?format=json&q={s}', 
                       zoom=5,
                       property_name='display_name',
                       marker=marker
                      )

dynamicMap.add_control(search)

widget_control1 = WidgetControl(widget=averageHomePricePlot, position='bottomleft')

dynamicMap.add_control(widget_control1)

with open('Neighborhoods.geojson') as f:
    neighborhoods = json.load(f)
    
geo = GeoJSON(data=neighborhoods, style={'fillColor': 'white', 'weight': 0.5}, hover_style={'fillColor': '#1f77b4'}, name='Neighborhoods')
dynamicMap.add_layer(geo)


control = LayersControl(position='bottomright')

def on_hover(event, feature, **kwargs):
    global neighborhood_name

    neighborhood = feature['properties']['nbhdname']
    update_figure(neighborhood, priceData)
            
def on_click(event, feature, **kwargs):
    neighborhood = feature['properties']['nbhdname']
    for index in range(0, len(predictionDataDict)):
        if predictionDataDict[index]['Neighborhood'] == neighborhood:
            circle = Circle()
            circle.color = "blue"
            circle.fill_color = "yellow"
            circle.location = (predictionDataDict[index]['Lat'], predictionDataDict[index]['Lon'])
            circle.radius = 10
            if predictionDataDict[index]['Bought'] == 1:
                circle.color = "green"
            else:
                circle.color = "red"
            if predictionDataDict[index]['ProbBought'] >= 0.5:
                circle.fill_color = "blue"
            else:
                circle.fill_color = "orange"
            dynamicMap.add_layer(circle)
            
            
dynamicMap.add_control(FullScreenControl())
dynamicMap.add_control(control)

                
geo.on_hover(on_hover)
geo.on_click(on_click)
dynamicMap

Map(center=[42.88023, -78.878738], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

In [18]:
#please use this cell to demonstrate example control of 
with open('Neighborhoods.geojson') as f:
    neighborhoods = json.load(f)

basicMap = Map(center = [42.880230, -78.878738], zoom = 11)
background = basemaps.OpenStreetMap.Mapnik
basicMap.add_control(FullScreenControl())
layers = []
widgets = []

@interact
def changeMap(Maps=['None','Basic Map', 'Clear']):
    if Maps == 'Clear':
        if layers != []:
            basicMap.clear_layers()
            layers.clear()
        basicMap.add_layer(background)
        layers.append('Background')
    elif Maps == 'Basic Map':
        if 'Geo' in layers:
            basicMap.remove_layer(basicMapGeoData)
        basicMapGeoData = GeoJSON(data=neighborhoods, style={'fillColor': 'white', 'weight': 0.5}, hover_style={'fillColor': '#1f77b4'}, name='Neighborhoods')
        basicMap.add_layer(basicMapGeoData)
        layers.append('Geo')
        
@interact
def changeFigure(Figures=['None', 'Average Home Price', 'Foreclosure', 'Clear']):
    global plotWidget
    if Figures != 'Clear' and Figures != 'None':
        if 'Geo' in layers:
            if len(widgets) == 0:
                plotWidget = WidgetControl(widget=plotDict[Figures], position='bottomleft', max_width=700, max_height=700)
                basicMap.add_control(plotWidget)
                widgets.append(plotDict[Figures])
    if Figures == 'Clear':
        basicMap.remove_control(plotWidget)
        widgets.clear()
        
# @interact
# def mouseEffects(Effects=['None', 'Hover', 'Click', 'Clear']):
#         if Effects == "Hover":
#             if 'Geo' in layers:
#                 basicMapGeoData = GeoJSON(data=neighborhoods, style={'fillColor': 'white', 'weight': 0.5}, hover_style={'fillColor': '#1f77b4'}, name='Neighborhoods')
#                 basicMap.add_layer(basicMapGeoData)
#                 basicMapGeoData.on_hover(on_hover)
#         elif Effects == "Click":
#             if 'Geo' in layers:
#                 basicMapGeoData = GeoJSON(data=neighborhoods, style={'fillColor': 'white', 'weight': 0.5}, hover_style={'fillColor': '#1f77b4'}, name='Neighborhoods')
#                 basicMap.add_layer(basicMapGeoData)
#                 basicMapGeoData.on_hover(on_click)
        
            
basicMap

interactive(children=(Dropdown(description='Maps', options=('None', 'Basic Map', 'Clear'), value='None'), Outp…

interactive(children=(Dropdown(description='Figures', options=('None', 'Average Home Price', 'Foreclosure', 'C…

Map(center=[42.88023, -78.878738], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…